In [2]:
%load_ext autoreload
%autoreload 2

In [7]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import bayesflow as bf

from data_generator import SSMDataGenerator

In [8]:
import bayesflow as bf

## Priors

Next, we will sample the DDM parameters from our simulator as priors from some distributions. Below, we specify the prior distributions:

\begin{equation}
v \sim \mathrm{Gamma}(2, 0.8), \; a \sim \mathrm{Gamma}(10, 0.2), \; z \sim \mathrm{Beta}(2, 2), \; t \sim \mathrm{Gamma}(1.5, 0.2).
\end{equation}

In [10]:
def priors(rng=None):

    if rng is None:
        rng = np.random.default_rng()

    drift = rng.gamma(2, 0.8)
    threshold = rng.gamma(10, 0.2)
    bias = rng.beta(2, 2)
    ndt = rng.gamma(1.5, 0.2)

    return dict(v=drift, a=threshold, z=bias, t=ndt)

We can inspect the sampled parameters as a dictionary.

In [11]:
theta = priors()
theta

{'v': 2.5976791951517626,
 'a': 1.9979524689332,
 'z': 0.3946465209616161,
 't': 0.14112064645132746}

## Simulator

For the simulator, we would use the SSM Data Generator as implemented in the previous workshop.

In [12]:
basic_ddm_generator = SSMDataGenerator(
    model="ddm",
    n_participants=10,
    n_trials=200
)

In [13]:
basic_ddm_df, basic_ddm_group_params, _ = \
    basic_ddm_generator.make_simple_hierarchical_dataset()

Attaching subject level parameters
Adding trials to attached df
Attaching simulations to df


In [14]:
basic_ddm_df

,v,a,z,t,beta_a_intercept_mean,beta_a_intercept_std,beta_a_intercept_subject,beta_t_intercept_mean,beta_t_intercept_std,beta_t_intercept_subject,beta_v_intercept_mean,beta_v_intercept_std,beta_v_intercept_subject,beta_z_intercept_mean,beta_z_intercept_std,beta_z_intercept_subject,participant_id,trial,rt,response
0,0.672842,1.279907,0.519003,0.924856,1.305819,0.114849,1.279907,0.882012,0.075765,0.924856,0.474907,0.323301,0.672842,0.503715,0.052438,0.519003,0,1,1.631770,1
1,0.672842,1.279907,0.519003,0.924856,1.305819,0.114849,1.279907,0.882012,0.075765,0.924856,0.474907,0.323301,0.672842,0.503715,0.052438,0.519003,0,2,1.475531,1
2,0.672842,1.279907,0.519003,0.924856,1.305819,0.114849,1.279907,0.882012,0.075765,0.924856,0.474907,0.323301,0.672842,0.503715,0.052438,0.519003,0,3,1.538448,1
3,0.672842,1.279907,0.519003,0.924856,1.305819,0.114849,1.279907,0.882012,0.075765,0.924856,0.474907,0.323301,0.672842,0.503715,0.052438,0.519003,0,4,1.679541,1
4,0.672842,1.279907,0.519003,0.924856,1.305819,0.114849,1.279907,0.882012,0.075765,0.924856,0.474907,0.323301,0.672842,0.503715,0.052438,0.519003,0,5,1.975137,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,-0.171075,1.139729,0.469068,0.871356,1.305819,0.114849,1.139729,0.882012,0.075765,0.871356,0.474907,0.323301,-0.171075,0.503715,0.052438,0.469068,9,196,2.128723,-1
1996,-0.171075,1.139729,0.469068,0.871356,1.305819,0.114849,1.139729,0.882012,0.075765,0.871356,0.474907,0.323301,-0.171075,0.503715,0.052438,0.469068,9,197,1.330640,1
1997,-0.171075,1.139729,0.469068,0.871356,1.305819,0.114849,1.139729,0.882012,0.075765,0.871356,0.474907,0.323301,-0.171075,0.503715,0.052438,0.469068,9,198,1.488002,-1
1998,-0.171075,1.139729,0.469068,0.871356,1.305819,0.114849,1.139729,0.882012,0.075765,0.871356,0.474907,0.323301,-0.171075,0.503715,0.052438,0.469068,9,199,2.613562,1
